In [13]:
import nltk
import numpy
import string
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.text import Text

import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ezeyn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ezeyn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# these are the companies listed on the Johannesburg Stock Exchange
listed_companies = pd.read_excel('Complete-List-of-Listed-Companies-on-South-Africa-Johannesburg-Stock-Exchange-Jan-2021.xlsx')
listed_companies.head()

,S.No.,Company Name,Ticker,Sector
0,1,4Sight Holdings Limited,4SI,Financial Services
1,2,ABSA Bank Limited,ABSP,Banks
2,3,Barclays Africa Group Limited,ABG.ZA,Banks
3,4,Accelerate Property Fund Limited,APF,Real Estate Investment Trusts
4,5,Accentuate Limited,ACE,Chemicals


In [92]:
# Taking unqiue company names into a list
companies = list(listed_companies['Company Name'].unique())

lowercased_companies = [company.lower() for company in companies]

In [16]:
# load the Zondo Comission Into State Capture reports
zondo_reports = pd.read_csv('zondo_reports.csv')
zondo_reports = zondo_reports.drop(columns=['Unnamed: 0'])
zondo_reports.head()

,report,text
0,OCR version - State Capture Commission Report ...,dicial \r\nn of Ir• \r\nS a \r\nI I eport: Par...
1,OCR version - State Capture Commission Report ...,Judicial Commission \r\nOf \r\n Inquiry ...
2,OCR version - State Capture Commission Report ...,\r\n \r\nJudicial Commission \r\nof \r\nInqu...
3,OCR version - State Capture Commission Report ...,Chairperson: Justice RtM Zondo \r\nActing Chie...
4,OCR version - State Capture Commission Report ...,\r\n \r\nJudicial Commission \r\nof \r\nInqu...


In [17]:
# Joining all reports together
reports = " ".join(zondo_reports['text'])

In [18]:
# Tokenizing the reports
reports_tokens = word_tokenize(reports)

In [169]:
# Formualting a manual list consisting of additional unnecessary words to be eliminated
junk_tokens = ['mr','ms','dr','p','``', '\'s','’','’’','\'\'','“','”','................................','l','would','also']

# Creating a final list of words to be removed from the tokens
removables = set(stopwords.words('English') + list(string.punctuation) + list(string.digits) + junk_tokens)

# Removing the unnecessary words
filtered_tokens = [token for token in reports_tokens if token not in removables]

In [171]:
# importing frequencies distribution
from nltk.probability import FreqDist

In [172]:
# forming a frequencies variable
frequencies = FreqDist(filtered_tokens)

In [173]:
reports_corpus = Text(filtered_tokens)

In [24]:
# importing a set consisting of English words
from english_words import english_words_set

In [226]:
# forming a list of common words to be removed from the company name parts: one of the list is manually created by visually
# observing the data values
additional_words_to_remove = ['south','e','new','group','value','african','go',
                'bid','transaction','total','w','capital','standard',
               'city','international','life','old','resource','old','global','central',
               'mine','trans','rand','great','distribution','huge','sun','advanced','premier',
               'pan','master','multichoice','eastern','efficient','union','net','freedom','middle',
                   'london','alexander','ab','british']

additional_words_to_remove_upper = [word.title() for word in additional_words_to_remove]
english_words_set_up = set([english_word.title() for english_word in english_words_set])

common_words = set(list(english_words_set_up) + list(string.ascii_uppercase) +
                      additional_words_to_remove_upper)

# first words in the names of companies
first_names_of_companies = []

for company in companies:
    name_parts = company.split(sep=' ')
    if name_parts[0] not in common_words:
        first_names_of_companies.append(name_parts[0])

# dictionary for each first word and its respective frequency
first_name_comp_freqs = {}

for company_name in first_names_of_companies:
    first_name_comp_freqs[company_name] = frequencies[company_name]

In [227]:
# dataframe that has two columns: company names (first word only) and its frequency
firt_name_comp_freqs_df = pd.DataFrame(list(zip(first_name_comp_freqs.keys(),first_name_comp_freqs.values())),columns=['company','frequency'])

# dataframe displaying values with above 1 frequency in a sorted form
firt_name_comp_freqs_df[firt_name_comp_freqs_df['frequency'] > 0].sort_values(by='frequency',ascending=False)

,company,frequency
198,SA,278
95,Glencore,190
155,Nedbank,114
80,EOH,107
151,MultiChoice,67
1,ABSA,29
117,Investec,14
175,Phumelela,13
205,Sasol,11
224,Telkom,10


In [132]:
stop_comp_words = [' Limited',' Holdings',' Plc',' Ltd',' Fund',' Group',' Company',' Corporation',' Corp']
filtered_companies = []

                
for company in companies:
    for stop in stop_comp_words:
        company = company.replace(stop,'')
    filtered_companies.append(company)

### Bigrams

In [163]:
tuple2 = []
tuple3 = []

for company in filtered_companies:
    
    words = company.split(sep=' ')
      
    if len(words) > 1:
            tuple2.append((words[0], words[1]))


In [164]:
bigrams = list(nltk.bigrams(filtered_reports_tokens)) 

In [165]:
for tuple_item in tuple2:
    if tuple_item in bigrams:
        print(f'{tuple_item} is mentioned')

('ABSA', 'Bank') is mentioned
('Barclays', 'Africa') is mentioned
('African', 'Bank') is mentioned
('Anglo', 'American') is mentioned
('Anglo', 'American') is mentioned
('Blue', 'Label') is mentioned
('Caxton', 'CTP') is mentioned
('City', 'Lodge') is mentioned
('Harmony', 'Gold') is mentioned
('Impala', 'Platinum') is mentioned
('Middle', 'East') is mentioned
('Rand', 'Merchant') is mentioned
('Royal', 'Bafokeng') is mentioned
('Standard', 'Bank') is mentioned
('Telkom', 'SA') is mentioned
('Trans', 'Hex') is mentioned
